In [1]:
import sqlite3
conn = sqlite3.connect("RH.db")

In [2]:
import csv
cur = conn.cursor()
with open("../recursos_humanos2.csv") as f:
    reader = csv.reader(f)
    data = list(reader)

In [3]:
#Creacion de la tabla con archivo csv.

cur.execute("""
    CREATE TABLE IF NOT EXISTS Detalle (
        satisfaction_level REAL,
        last_evaluation REAL,
        number_project INTEGER,
        average_montly_hours INTEGER,
        time_spend_company INTEGER,
        Work_accident INTEGER,
        left INTEGER,
        promotion_last_5years INTEGER,
        sales TEXT,
        salary TEXT
    )
""")

In [4]:
#incersion de los datos en la tabla

with open("../recursos_humanos2.csv", "r") as f:
    reader = csv.reader(f)
    next(reader)  # Saltar la cabecera
    for row in reader:
        cur.execute("""
            INSERT INTO Detalle (
                satisfaction_level,
                last_evaluation,
                number_project,
                average_montly_hours,
                time_spend_company,
                Work_accident,
                left,
                promotion_last_5years,
                sales,
                salary
            ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
        """, row)

# Guardar los cambios
conn.commit()


In [5]:
cur.execute("SELECT AVG(satisfaction_level) FROM Detalle")
average_satisfaction = cur.fetchone()[0]

# Guardar los cambios
conn.commit()

average_satisfaction

0.6128335222348156

In [6]:
cur.execute("SELECT AVG(satisfaction_level) FROM Detalle WHERE left = 1")
average_satisfaction_left = cur.fetchone()[0]

# Obtener el promedio de satisfacción para empleados que se quedaron en la empresa
cur.execute("SELECT AVG(satisfaction_level) FROM Detalle WHERE left = 0")
average_satisfaction_stay = cur.fetchone()[0]


average_satisfaction_left, average_satisfaction_stay

(0.44009801176141133, 0.666809590479524)

In [7]:
# Obtener el número promedio de horas de trabajo para empleados con salario "low" o "medium"
cur.execute("""
    SELECT AVG(average_montly_hours)
    FROM Detalle
    WHERE salary IN ('low', 'medium')
""")
average_hours_low_medium = cur.fetchone()[0]


average_hours_low_medium

201.15666327568667

In [11]:
# Obtener registros de empleados que obtuvieron una promoción en los últimos 5 años y abandonaron la empresa
cur.execute("""
    SELECT *
    FROM Detalle
    WHERE promotion_last_5years = 1 AND left = 1
""")
promoted_and_left = cur.fetchall()


import pandas as pd

# Convertir los resultados en un DataFrame para mejor visualización
columns = [
    "satisfaction_level", "last_evaluation", "number_project", 
    "average_montly_hours", "time_spend_company", "Work_accident", 
    "left", "promotion_last_5years", "sales", "salary"
]
df_promoted_and_left = pd.DataFrame(promoted_and_left, columns=columns)
df_promoted_and_left

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,sales,salary
0,0.45,0.51,2,160,3,1,1,1,sales,low
1,0.79,0.59,4,139,3,0,1,1,management,low
2,0.41,0.46,2,160,3,0,1,1,sales,low
3,0.11,0.79,6,292,4,0,1,1,technical,low
4,0.41,0.56,2,154,3,0,1,1,support,medium
...,...,...,...,...,...,...,...,...,...,...
71,0.45,0.51,2,160,3,1,1,1,sales,low
72,0.79,0.59,4,139,3,0,1,1,management,low
73,0.41,0.46,2,160,3,0,1,1,sales,low
74,0.11,0.79,6,292,4,0,1,1,technical,low


In [12]:
# Obtener registros de empleados que obtuvieron una calificación de 0.9 o más en su última evaluación
cur.execute("""
    SELECT *
    FROM Detalle
    WHERE last_evaluation >= 0.9
""")
high_evaluation = cur.fetchall()

# Cerrar la conexión
conn.close()

# Convertir los resultados en un DataFrame para mejor visualización
df_high_evaluation = pd.DataFrame(high_evaluation, columns=columns)
df_high_evaluation

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,sales,salary
0,0.89,1.00,5,224,5,0,1,0,sales,low
1,0.84,0.92,4,234,5,0,1,0,sales,low
2,0.78,0.99,4,255,6,0,1,0,sales,low
3,0.09,0.95,6,304,4,0,1,0,sales,low
4,0.89,0.92,5,242,5,0,1,0,sales,low
...,...,...,...,...,...,...,...,...,...,...
11947,0.11,0.97,6,310,4,0,1,0,accounting,medium
11948,0.09,0.93,6,296,4,0,1,0,technical,medium
11949,0.73,0.93,5,162,4,0,1,0,technical,low
11950,0.91,0.99,5,254,5,0,1,0,technical,medium
